In [5]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.svm import SVC

from math import nan

from sklearn.base import BaseEstimator, TransformerMixin
from imblearn.pipeline import Pipeline 

pd.set_option('display.max_columns', None)

In [ ]:
path = '../data/processed/combined_data.csv'
df = pd.read_csv(path, index_col='date')


,Ferrero.1,Mondelez.1,LindtSpruengli.1
date,,,
2019-01-31,NaN,39.095000,NaN
2019-02-28,NaN,41.495000,NaN
2019-03-31,NaN,42.429048,NaN
2019-04-30,NaN,44.421250,NaN
2019-05-31,NaN,46.047045,NaN
2019-06-30,NaN,47.868158,NaN
2019-07-31,NaN,48.906739,NaN
2019-08-31,NaN,48.591818,NaN
2019-09-30,NaN,50.056667,NaN


In [22]:
y = df.iloc[:,2]
mask_f = ~df.loc[:,'Ferrero.1'].isna()
#pd.DataFrame(y).loc[mask_f ,:]
y.loc[mask_f]


date
2021-04-30    469856.0
2021-05-31    471102.0
2021-06-30    475148.0
2021-07-31    478023.0
2021-08-31    480318.0
2021-09-30    480784.0
2021-10-31    479685.0
2021-11-30    479855.0
2021-12-31    476249.0
2022-01-31    471444.0
2022-02-28    474845.0
2022-03-31    476996.0
2022-04-30    476053.0
2022-05-31    477141.0
2022-06-30    480472.0
2022-07-31    482441.0
2022-08-31    484799.0
2022-09-30    484855.0
2022-10-31    483642.0
2022-11-30    484602.0
2022-12-31    481989.0
2023-01-31    478258.0
2023-02-28    478175.0
2023-03-31    482368.0
2023-04-30    480562.0
2023-05-31    482328.0
2023-06-30    485636.0
2023-07-31    487498.0
2023-08-31    490757.0
2023-09-30    491796.0
2023-10-31    492176.0
2023-11-30    493135.0
2023-12-31    489662.0
2024-01-31    489350.0
2024-02-29    492620.0
2024-03-31    493844.0
2024-04-30    495003.0
2024-05-31    494753.0
2024-06-30    496660.0
2024-07-31    499550.0
2024-08-31    501531.0
2024-09-30    501671.0
2024-10-31    501229.0
2024-1

In [ ]:
class DataImputation(TransformerMixin, BaseEstimator):
    """Imputs missing values. """
    
    def __init__(self, model, columns):
        """Initalizes the attributes."""
        self.features = []
        self.num_cols = []
        self.dict_num = {}
        self.model_start = model.copy()
        self.model_middle = model.copy()
        self.model_end = model.copy()
        self.columns = columns

    def fit(self, X, y=None):
        mask_f = X.loc[:,'Ferrero.1'].isna()
        #mask_m = ~X.loc[:,'Mondelez.1'].isna()
        mask_l = X.loc[:,'LindtSpruengli.1'].isna()
        feat_start = X.[:, columns + ['Mondelez.1']]#.loc[mask_f &  mask_l, :]
        target_start = y#.loc[mask_f &  mask_l]
        model_start.fit(feat_start, target_start)

        feat_middle = X.loc[~mask_f , :]
        target_middle = y.loc[~mask_f]
        feat_middle['target_pred_by_1'] = model_start.predict(feat_middle)

        model_middle.fit( feat_middle.loc[:,['target_pred_by_1', 'Ferrero.1']],target_middle)

        feat_end = X.loc[~mask_f & ~mask_l, :]
        target_end = y.loc[~mask_f & ~mask_l]
        feat_end['target_pred_by_2'] = model_middle.predict(feat_end)

        model_end.fit(feat_end.loc[:,['target_pred_by_2', 'LindtSpruengli.1']],target_end)

        self.model_start = model_start
        self.model_middle = model_middle
        self.model_end = model_end

        return self
    
    def transform(self, X, y=None):
        """Fill the missing values and drop some columns.
        
        
        """
        df = X.copy()

        self.features = df.columns.values
        return df
    
    def get_feature_names_out(self, input_features=None):
        """Returns the names of the features."""
        return self.features

In [ ]:
target = 'Wert der zum Absatz bestimmten Produktion_Schokolade u.a. kakaohaltige Lebensmittelzubereit.'
number_train = 70
# the dict has entry of the form column: [lag_0, lag_1,lag_2]
# note that the column must be given by the name and not the number and that if the 
# column should be kept it need to have lag = 0
dict_lag = {}
for k in [39, 41, 43, 45, 78, 88, 93]:
    dict_lag.update({df.columns.values[k]: [1,2,3]})
for k in [166, 167, 168, 169, 170]: 
    dict_lag.update({df.columns.values[k]: [0]})

df_selected_col = df.loc[:, list(dict_lag)]
df_train = df_selected_col.iloc[:number_train, :]
df_test = df_selected_col.iloc[number_train: , :]
df_train

pipeline = Pipeline([('DaIm', DataImputation()), 
                     ('FE', FeatureEngineer(dict_lag))])

df_train_transf = pipeline.fit_transform(df_train)
df_test_transf = pipeline.transform(df_test)
df_train_transf